In [ ]:
!pip install pretrainedmodels
import pretrainedmodels as pm

In [ ]:
import os
from torch.utils.data import Dataset
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import time
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import datetime as dt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import torch.nn as nn


# from fastai.vision import *
# import fastai
# from PIL import Image
# import shutil
# import fastai
# from fastai import vision

#!pip install pretrainedmodels
#mport pretrainedmodels 
%matplotlib inline

In [ ]:
data_path = "../input/ammi-2020-convnets/train/train"
test_path = "../input/ammi-2020-convnets/test/test"
extraimage_path = "../input/ammi-2020-convnets/extraimages/extraimages"

In [ ]:
os.listdir(data_path)

In [ ]:
os.listdir(data_path)[:10]

In [ ]:
# %matplotlib inline
# from matplotlib import pyplot as plt
# import cv2
# image=cv2.imread('data_path/cmd/train-cmd-1992.jpg')
# plt.imshow(image)
# plt.show()

In [ ]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(448),
                                       transforms.RandomVerticalFlip(),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(448),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                      # transforms.CenterCrop(244),
                                       transforms.ToTensor()])
# train_transforms = transforms.Compose([transforms.RandomRotation(30),
#                                        transforms.RandomResizedCrop(448),
#                                        transforms.RandomHorizontalFlip(),
#                                        transforms.ToTensor()])

# test_transforms = transforms.Compose([ transforms.Resize(255),
#                                        transforms.CenterCrop(224),
#                                        transforms.ToTensor()])

normalize = transforms.Normalize(mean=mean, std=std)

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
            
        return im.view(3, 448, 448), classCategory

In [ ]:
train_data = CassavaDataset(data_path, transform=train_transforms)
test_data = CassavaDataset(test_path, transform=test_transforms)

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

In [ ]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=8,
                                             sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=8,
                                             sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8)

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# # Define Models 

# model = models.resnet50(pretrained=True).to(device)
    
# for param in model.parameters():
#      param.requires_grad = True   
    
# model.fc = nn.Sequential(
#                nn.Linear(2048, 128),# 50176
#                nn.ReLU(inplace=True),
#                nn.Linear(128, 5)).to(device)
# if torch.cuda.is_available():
#     model = model.cuda()

In [ ]:
# def se_resnext50_32x4d(pretrained=True):
#     pretrained = 'imagenet' if pretrained else None
#     model = pretrainedmodels.se_resnext50_32x4d(pretrained=pretrained)
#     return model

# model = se_resnext50_32x4d(pretrained=True)

In [ ]:
# for param in model.parameters():
#     param.requires_grad = True

# # new final layer with 5 classes
# model.avg_pool = torch.nn.AdaptiveAvgPool2d(1)
# num_ftrs = model.last_linear.in_features
# model.last_linear = torch.nn.Linear(num_ftrs, 5)

# if torch.cuda.is_available():
#     model = model.cuda()

In [ ]:
# model = models.resnext101_32x8d(pretrained=True).to(device)
# #model = models.squeezenet1_1(pretrained=True).to(device)
    
# for param in model.parameters():
#      param.requires_grad = True   
    
# model.fc = nn.Sequential(
#                nn.Linear(model.fc.in_features, 128),# 50176
#                nn.Dropout(0.1),
#                nn.ReLU(inplace=True),
#                nn.Linear(128, 5)).to(device)
# if torch.cuda.is_available():
#     model = model.cuda()

In [ ]:
 class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB, modelC, nb_classes=5):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.modelC = modelC
        # Remove last linear layer
        self.modelA.fc = nn.Identity()
        self.modelB.fc = nn.Identity()
        self.modelC.fc = nn.Identity()
        
        # Create new classifier
        self.classifier = nn.Linear(2048+2048+512, nb_classes)
        
    def forward(self, x):
        x1 = self.modelA(x.clone())  # clone to make sure x is not changed by inplace methods
        x1 = x1.view(x1.size(0), -1)
        x2 = self.modelB(x)
        x2 = x2.view(x2.size(0), -1)
        
        x3 = self.modelC(x)
        x3 = x3.view(x3.size(0), -1)
        x = torch.cat((x1, x2,x3), dim=1)
        x = self.classifier(F.relu(x))
        return x

# Train your separate models
# ...
# We use pretrained torchvision models here
modelA = models.resnext101_32x8d(pretrained=True)
modelB = model.resnet50(pretrained=True)
modelC = models.resnet18(pretrained=True)

# Freeze these models
for param in modelA.parameters():
    param.requires_grad_(False)

for param in modelB.parameters():
    param.requires_grad_(False)
    
for param in modelC.parameters():
    param.requires_grad_(False)

# Create ensemble model
model = MyEnsemble(modelA, modelB, modelC).to(device)
if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
# def se_resnext101_32x4d(pretrained=True):
#     pretrained = 'imagenet' if pretrained else None
#     model = pm.se_resnext101_32x4d(pretrained=pretrained)
#     return model

#     model = se_resnext101_32x4d(pretrained=True)

In [ ]:
#criterion = nn.NLLLoss()

criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
#optimizer = torch.optim.Adam(model.fc.parameters())
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# this function is used during training process, to calculation the loss and accuracy
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
# def train(model, criterion, data_loader, optimizer,exp_lr_scheduler, num_epochs):
#     """Simple training loop for a PyTorch model.""" 
    
#     # Make sure model is in training mode.
#     model.train()
    
#     # Move model to the device (CPU or GPU).
#     model.to(device)
    
#     # Exponential moving average of the loss.
#     ema_loss = None
    
#     # Loop over epochs.
#     for epoch in range(num_epochs):
        
#       # Loop over data.
#       for batch_idx, (data, target) in enumerate(data_loader):
            
#           # Forward pass.
#           output = model(data.to(device))#.to(device)
#           loss = criterion(output.to(device), target.to(device))
          
#           # Backward pass.
#           optimizer.zero_grad()
#           loss.backward()
#           optimizer.step()
          
#           # NOTE: It is important to call .item() on the loss before summing.
#           if ema_loss is None:
#             ema_loss = loss.item()
#           else:
#             ema_loss += (loss.item() - ema_loss) * 0.01 
            
#       exp_lr_scheduler.step()
          
#       # Print out progress the end of epoch.
#       print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, ema_loss),)

In [ ]:
# total_loss_train, total_acc_train = [],[]
# def train( model, criterion,train_loader, optimizer,exp_lr_scheduler, epoch):
#     model.train()
#     train_loss = AverageMeter()
#     train_acc = AverageMeter()
#     curr_iter = (epoch - 1) * len(train_loader)
#     for i, data in enumerate(train_loader):
#         images, labels = data
#         N = images.size(0)
#         # print('image shape:',images.size(0), 'label shape',labels.size(0))
#         images = Variable(images).to(device)
#         labels = Variable(labels).to(device)

#         optimizer.zero_grad()
#         outputs = model(images)

#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         prediction = outputs.max(1, keepdim=True)[1]
#         train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
#         train_loss.update(loss.item())
#         curr_iter += 1
#         if (i + 1) % 100 == 0:
#             print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
#                 epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
#             total_loss_train.append(train_loss.avg)
#             total_acc_train.append(train_acc.avg)
#     return train_loss.avg, train_acc.avg



total_step = len(train_loader)
def train(model, criterion, data_loader, optimizer,scheduler, num_epochs): # scheduler,
    """Simple training loop for a PyTorch model.""" 
    
    # Make sure model is in training mode.
    model.train()
    
    # Move model to the device (CPU or GPU).
    model.to(device)
    
    # Exponential moving average of the loss.
    ema_loss = None
    
    # Loop over epochs.
    for epoch in range(num_epochs):
        
            for i, (images, labels) in enumerate(data_loader):
                images = images.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
            scheduler.step()
        
            #if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
# def test(model, data_loader):
#     """Measures the accuracy of a model on a data set.""" 
#     # Make sure the model is in evaluation mode.
#     model.eval()
#     correct = 0

#     # We do not need to maintain intermediate activations while testing.
#     with torch.no_grad():
        
#         # Loop over test data.
#         for data, target in data_loader:
          
#             # Forward pass.
#             output = model(data.to(device))
            
#             # Get the label corresponding to the highest predicted probability.
#             pred = output.argmax(dim=1, keepdim=True)
            
#             # Count number of correct predictions.\
#             correct += pred.cpu().eq(target.view_as(pred)).sum().item()

#     # Print test accuracy.
#     percent = 100. * correct / int(len(train_data)*validation_split)#len(data_loader.dataset)
#     #print(f'Accuracy: {correct} / {len(data_loader.dataset)} ({percent:.0f}%)')
#     print(f'Accuracy: {correct} / {int(len(train_data)*validation_split)} ({percent:.0f}%)')
#     torch.save(model.state_dict(), 'model.ckpt')
#     return percent
    

In [ ]:
def validate(val_loader, model, criterion, optimizer, num_epochs):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (num_epochs, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    torch.save(model.state_dict(), 'model.ckpt')
    return val_loss.avg, val_acc.avg

In [ ]:
 train(model, criterion, train_loader, optimizer,exp_lr_scheduler, 10)


In [ ]:

validate(valid_loader, model, criterion, optimizer, 10)


In [ ]:
# model.avg_pool = torch.nn.AdaptiveAvgPool2d(1)
# num_ftrs = model.last_linear.in_features
# model.last_linear = torch.nn.Linear(num_ftrs, 5)
# if torch.cuda.is_available():
# model = model.cuda()# train(model, criterion, train_loader, optimizer,exp_lr_scheduler, num_epochs=25)
# # test(model, valid_loader)#test_loader

In [ ]:
#load saved model to make predictions on test data
model.load_state_dict(torch.load('model.ckpt'))
model.eval()


In [ ]:
# epochs = range(1, len(total_acc_train) + 1)
# #Train and validation accuracy
# plt.plot(epochs, total_acc_train, 'b', label='Train accurarcy')
# #plt.plot(epochs, test_accuracy_list, 'r', label='Validation accurarcy')
# plt.title('Training and Validation accurarcy')
# plt.legend()

# plt.figure()
# #Train and validation loss
# plt.plot(epochs, total_loss_train 'b', label='Training loss')
# #plt.plot(epochs, validation_loss_list, 'r', label='Validation loss')
# plt.title('Training and Validation loss')
# plt.legend()
# plt.show()

In [ ]:
## Make Submission

In [ ]:
def process_image(image_dir):
# Process a PIL image for use in a PyTorch model
# tensor.numpy().transpose(1, 2, 0)
    image = Image.open(image_dir)
    preprocess = transforms.Compose([ transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor()])
    image = preprocess(image)
  # Convert 2D image to 1D vector
    image = np.expand_dims(image, 0)
    image = torch.from_numpy(image)
    inputs = image.to(device)
    return inputs

In [ ]:
# Using our model to predict the label
def predict(image, model):
# Pass the image through our model
    output = model(image)
# Reverse the log function in our output
    output = torch.exp(output)
# Get the top predicted class, and the output percentage for
# that class
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

In [ ]:
class_names = train_loader.dataset.classes

In [ ]:
test_directory = '../input/ammi-2020-convnets/test/test/0'

predictions, test_image_fileName = [], []
try:
    test_images = listdir(test_directory)
    for images in test_images:
        test_image_fileName.append(images)
        image = process_image(f'{test_directory}/{images}')
        top_prob, top_class = predict(image, model)
        predictions.append(class_names[top_class])
except Exception as e:
    print(e)